# Non-linear Matrix Factorization with Gaussian Processes

This is an attempt to reproduce [this ICML 2009 paper](http://people.ee.duke.edu/~lcarin/MatrixFactorization.pdf).

In [1]:
import tensorflow as tf
DEVICE = "/cpu:0"

In [6]:
# Simulated data
import numpy as np

N = 3  # items
D = 10  # users
q = 2  # latent dimension
true_alpha_x = 0.05
true_alpha_w = 0.02
true_sigma = 0.001

X = np.random.randint(50, size=(N, q))
W = np.random.normal(0, true_alpha_w, (D, q))
rating_rows = []
for i in range(N):
    rating_rows.append(np.random.multivariate_normal(W.dot(X[i, :]), true_sigma ** 2 * np.eye(D)))
Y = np.array(rating_rows)
Y.shape

(3, 10)

In [7]:
from scipy.sparse import coo_matrix

dataset = coo_matrix(Y)

In [8]:
# Prepare batches
from sklearn.model_selection import ShuffleSplit

splitter = ShuffleSplit(n_splits=1, test_size=0.2)
for i_train, i_test in splitter.split(dataset.data):
    train = coo_matrix((dataset.data[i_train], (dataset.row[i_train], dataset.col[i_train])), shape=(N, D)).tocsc()
    test = coo_matrix((dataset.data[i_test], (dataset.row[i_test], dataset.col[i_test])), shape=(N, D)).tocsc()

In [9]:
for user_id in range(5):
    # print(test[:, user_id].getnnz())
    print(test[:, user_id].indices)
    print(test[:, user_id].data)

[1 2]
[-1.0362237  -0.58233791]
[1]
[ 1.10123861]
[0]
[ 0.1269993]
[]
[]
[]
[]


In [10]:
user_batch = tf.placeholder(tf.int32, shape=[None], name="id_user")
item_batch = tf.placeholder(tf.int32, shape=[None], name="id_item")
rate_batch = tf.placeholder(tf.float32, shape=[None, None], name="rate")
pred_batch = tf.placeholder(tf.int32, shape=[None], name="id_pred")

In [11]:
def kernel(X, Y):
    return tf.matmul(X, tf.transpose(Y))

class MFGP:
    def __init__(self):
        with tf.device(DEVICE):
            with tf.variable_scope("conv1"):
                self.sigma = tf.Variable(tf.random_normal([1]), name="sigma")
                self.alpha_x = tf.Variable(tf.random_normal([1]), name="alpha_x")
                self.alpha_w = tf.Variable(tf.random_normal([1]), name="alpha_w")
                self.w_user = tf.Variable(tf.random_normal([D, q], stddev=0.2), name="embd_user")
                self.x_item = tf.Variable(tf.random_normal([N, q], stddev=0.2), name="embd_item")
        # embd_user = tf.nn.embedding_lookup(w_user, user_batch, name="embedding_user")
        # embd_item = tf.nn.embedding_lookup(x_item, item_batch, name="embedding_item")

    def debug(self, sess, item_batch):
        sess.run(init_op)
        print(sess.run(self.x_item))
        print(sess.run(item_batch))        

    def predict(self, user_batch, item_batch, rate_batch, pred_batch):
        with tf.device(DEVICE):
            x_batch = tf.nn.embedding_lookup(self.x_item, item_batch)
            x_pred = tf.nn.embedding_lookup(self.x_item, pred_batch)
            N_j = tf.shape(item_batch)[0]
            cov = kernel(x_batch, x_batch) + self.sigma ** 2 * tf.eye(N_j)
            s = tf.matmul(tf.matrix_inverse(cov), kernel(x_batch, x_pred))
            prediction = tf.matmul(tf.transpose(s), rate_batch)
        return tf.reshape(prediction, [-1])

    def get_loss(self, user_batch, item_batch, rate_batch):
        with tf.device(DEVICE):
            x_batch = tf.nn.embedding_lookup(self.x_item, item_batch)
            N_j = tf.shape(item_batch)[0]
            cov = 1/self.alpha_w * kernel(x_batch, x_batch) + self.sigma ** 2 * tf.eye(N_j)
            #loss3 = tf.matrix_inverse(cov)
            #loss2 = tf.matmul(tf.matrix_inverse(cov), rate_batch)
            loss = (tf.cast(N_j, tf.float32) * tf.log(tf.matrix_determinant(cov))
                    + tf.reshape(tf.matmul(tf.transpose(rate_batch), tf.matmul(tf.matrix_inverse(cov), rate_batch)), []))
        return loss

    def optimization(self, user_batch, item_batch, rate_batch, learning_rate=0.001):
        global_step = tf.train.get_global_step()
        assert global_step is not None
        with tf.device(DEVICE):
            # cost_l2 = tf.nn.l2_loss(tf.subtract(infer, rate_batch))
            # penalty = tf.constant(reg, dtype=tf.float32, shape=[], name="l2")
            # cost = tf.add(cost_l2, tf.multiply(regularizer, penalty))
            loss = self.get_loss(user_batch, item_batch, rate_batch)
            train_op = tf.train.MomentumOptimizer(learning_rate, 0.9, use_nesterov=True).minimize(loss, global_step=global_step)
        return loss, train_op

In [12]:
from collections import deque
import time

mfgp = MFGP()
prediction = mfgp.predict(user_batch, item_batch, rate_batch, pred_batch)

global_step = tf.contrib.framework.get_or_create_global_step()
loss, train_op = mfgp.optimization(user_batch, item_batch, rate_batch, learning_rate=0.01)

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    summary_writer = tf.summary.FileWriter(logdir="/tmp/log_tf", graph=sess.graph)
    print(" | ".join(["user_id", "epoch", "train_error", "val_error", "elapsed_time"]))
    errors = deque()
    start = time.time()
    for epoch in range(100):
        for user_id in range(D):
            train_batch = train[:, user_id]
            N_j = train_batch.getnnz()
            users, items, rates = [user_id] * N_j, train_batch.indices, train_batch.data
            _, pred = sess.run([train_op, prediction], feed_dict={user_batch: users,
                                                                  item_batch: items,
                                                                  rate_batch: np.array(rates).reshape(N_j, 1),
                                                                  pred_batch: items})
            errors.extend(np.power(pred - rates, 2))
            if user_id % 2 == 0 and epoch % 25 == 0:
                print('train', pred[:5], rates[:5])
                #print('all errors', errors)
                train_err = np.sqrt(np.mean(errors))
                test_err2 = np.array([])
                # for user_id in range(D):
                # train_batch = train[:]
                test_batch = test[:, user_id]
                preds = test_batch.indices
                truth = test_batch.data
                # true_pred = users, items, rates = [user_id] * N_j, test_batch.indices, test_batch.data, train
                this_error, pred = sess.run([loss, prediction], feed_dict={user_batch: users,
                                                                item_batch: items,
                                                                rate_batch: np.array(rates).reshape(len(rates), 1),
                                                                pred_batch: preds})
                print('test', pred[:5], truth[:5])
                all_errors = []
                for test_user_id in range(D):
                    test_batch = train[:, test_user_id]
                    test_N_j = test_batch.getnnz()
                    test_users, test_items, test_rates = np.array([test_user_id] * test_N_j), test_batch.indices, np.array(test_batch.data).reshape(test_N_j, 1)
                    this_user_error = sess.run(loss, feed_dict={user_batch: test_users, item_batch: test_items, rate_batch: test_rates})
                    all_errors.append(this_user_error)
                test_err2 = np.append(test_err2, np.power(pred - truth, 2))
                end = time.time()
                test_err = np.sqrt(np.mean(test_err2))
                if len(preds):
                    print("user={:2d} epoch={:3d} train_err={:f} test_err={:f} his_loss={:f}, sum_losses={:f}, all_losses={:s}".format(user_id, epoch, train_err, test_err,
                                                       this_error, sum(all_errors), str(all_errors)))
                else:
                    print('no test for user={:2d}'.format(user_id))
                start = end
    print('learned')
    print(mfgp.sigma.eval())
    print(mfgp.alpha_w.eval())

user_id | epoch | train_error | val_error | elapsed_time
train [-0.00015972] [-0.01218857]
test [ -4.26896499e-04  -3.86088650e-05] [-1.0362237  -0.58233791]
user= 0 epoch=  0 train_err=0.012029 test_err=0.840222 his_loss=-0.196617, sum_losses=-2.362280, all_losses=[-0.19661705, -0.22379178, 0.12683764, -0.21289325, -0.99248475, 0.71130639, -0.13664854, -0.49668658, -0.37515742, -0.566145]
train [ 0.11828238  0.0470581 ] [ 0.55765551  0.31935937]
test [ 0.03250591] [ 0.1269993]
user= 2 epoch=  0 train_err=0.353820 test_err=0.094493 his_loss=-3.330988, sum_losses=-60.031102, all_losses=[-1.3122668, -3.7542503, -3.3309882, -8.4445868, -10.412377, -1.7489336, -8.280901, -9.1877365, -4.2057743, -9.3532877]
train [-0.00161645 -0.07598279 -0.03295146] [-0.0175215  -0.08341457 -0.04742625]
test [] []
no test for user= 4
train [ 0.00172567  0.02016635  0.00294806] [ 0.0008417   0.82922637  0.4636775 ]
test [] []
no test for user= 6
train [-0.00929633  0.00928263] [-0.03902631  0.50912533]
test

/Users/jin/Sites/mangaki/venv/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jin/Sites/mangaki/venv/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


train [ -5.63762551e-05] [-0.01218857]
test [ -3.73272283e-04   2.81558077e-05] [-1.0362237  -0.58233791]
user= 0 epoch= 25 train_err=0.348839 test_err=0.840279 his_loss=10.832930, sum_losses=674.223742, all_losses=[10.83293, 43.329964, 43.603813, 97.904327, 97.904312, 43.605469, 97.904327, 97.90432, 43.32996, 97.90432]
train [ 0.19459763 -0.00166854] [ 0.55765551  0.31935937]
test [ 0.01042591] [ 0.1269993]
user= 2 epoch= 25 train_err=0.349171 test_err=0.116573 his_loss=43.602654, sum_losses=674.210590, all_losses=[10.83284, 43.329613, 43.602654, 97.902321, 97.902306, 43.604301, 97.902321, 97.902313, 43.329609, 97.902313]
train [-0.00162435 -0.02942814  0.00028506] [-0.0175215  -0.08341457 -0.04742625]
test [] []
no test for user= 4
train [ 0.01547016  0.28914541 -0.00246485] [ 0.0008417   0.82922637  0.4636775 ]
test [] []
no test for user= 6
train [-0.00135513  0.00081298] [-0.03902631  0.50912533]
test [-0.0041378] [ 0.91215863]
user= 8 epoch= 25 train_err=0.349291 test_err=0.91629